In [178]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

In [179]:
R = pd.read_csv("프롬데이원 1분기 최근 구매일 점수 .csv")
F = pd.read_csv("프롬데이원 1분기 주문수 점수 .csv")
M = pd.read_csv("프롬데이원 1분기 구매액 점수 .csv")

In [180]:
print(R.head())
print(R.shape)

                   주문자ID번호  M점수
0    rain3770010-3298-3375    1
1   sominnara010-6505-6625    1
2  33127561@n010-6857-8723    1
3  70359972@n010-3080-0528    1
4  silentna89010-6315-0720    1
(4774, 2)


In [181]:
print(F.head())
print(F.shape)

                  주문자ID번호  F점수
0  amber0504010-0000-0000    5
1  amanda123010-4786-6885    5
2       wood010-9599-9787    5
3     vminis010-8006-1617    5
4    sjin829010-3422-7118    5
(4774, 2)


In [182]:
print(M.head())
print(M.shape)

              주문자ID번호  R점수
0         오주영juyo****    1
1  spi41010-3397-5841    1
2       010-3383-9815    1
3         김현지happ****    1
4         박시은104t****    1
(4774, 2)


In [183]:
RF = pd.merge(R, F, on="주문자ID번호")
RFM = pd.merge(RF,M, on="주문자ID번호")

In [184]:
RFM.shape

(4774, 4)

In [185]:
RFM

,주문자ID번호,M점수,F점수,R점수
0,rain3770010-3298-3375,1,2,2
1,sominnara010-6505-6625,1,1,1
2,33127561@n010-6857-8723,1,2,1
3,70359972@n010-3080-0528,1,3,2
4,silentna89010-6315-0720,1,3,5
...,...,...,...,...
4769,emihui010-8369-5115,5,3,3
4770,김다예daye****,5,2,1
4771,wldmsd22010-2260-4430,5,4,5
4772,ohninipink010-9538-0114,5,4,4


In [186]:
RFM["RFM점수"] = RFM["M점수"]+RFM["F점수"]+RFM["R점수"]

In [187]:
# RFM["W_RFM점수"]  = 0.5 * RFM["M점수"] + 0.3 * RFM["F점수"] + 0.8* RFM["R점수"]

In [188]:
RFM[:10]

,주문자ID번호,M점수,F점수,R점수,RFM점수
0,rain3770010-3298-3375,1,2,2,5
1,sominnara010-6505-6625,1,1,1,3
2,33127561@n010-6857-8723,1,2,1,4
3,70359972@n010-3080-0528,1,3,2,6
4,silentna89010-6315-0720,1,3,5,9
5,1168491540@k010-9622-1190,1,2,2,5
6,40719331@n010-2506-9021,1,2,1,4
7,pianistangel010-4123-5494,1,2,1,4
8,ygo0416010-3510-7423,1,2,1,4
9,58662105@n010-6440-4759,1,3,4,8


In [189]:
RFM["RFM점수"].sort_values()

125      3
609      3
610      3
612      3
614      3
        ..
4292    15
4300    15
4626    15
3238    15
3688    15
Name: RFM점수, Length: 4774, dtype: int64

### 2분기 RFM 

In [190]:
R2 = pd.read_csv("프롬데이원 2분기 최근 구매일 점수.csv")
F2 = pd.read_csv("프롬데이원 2분기 주문수 점수.csv")
M2 = pd.read_csv("프롬데이원 2분기 구매액 점수.csv")

In [191]:
print(R2.head())
print(R2.shape)

                    주문자ID번호  R점수
0    6773359@n010-9381-5574    5
1   49938077@n010-8288-1034    5
2       plom90010-4209-3577    5
3             010-2926-1299    5
4  dlagmltmd95010-3205-7859    5
(4774, 2)


In [192]:
RF2 = pd.merge(R2, F2, on="주문자ID번호")
RFM2 = pd.merge(RF2,M2, on="주문자ID번호")

In [193]:
RFM2["RFM점수"] = RFM2["M점수"]+RFM2["F점수"]+RFM2["R점수"]

In [194]:
# RFM["W_RFM점수"]  = 0.5 * RFM["M점수"] + 0.3 * RFM["F점수"] + 0.8* RFM["R점수"]

In [195]:
RFM2.head()

,주문자ID번호,R점수,F점수,M점수,RFM점수
0,6773359@n010-9381-5574,5,3,4,12
1,49938077@n010-8288-1034,5,3,3,11
2,plom90010-4209-3577,5,4,4,13
3,010-2926-1299,5,3,1,9
4,dlagmltmd95010-3205-7859,5,2,2,9


## 가중치를 부여한 RFM점수 평균 제곱오차, 절대오차를 통해 정확도를 구하기

#### 주로 mae(절대오차)사용

In [196]:
#rfmc라는 복사본 만들기
rfmc=RFM.copy()

In [197]:
# 머신러닝을 실행하기 위해서는 int형 변수가 되어야하기 때문에 W_RFM을 구하고 나서 100을 곱하기
rfmc["W_RFM점수"]  = (0.03 * rfmc["M점수"] + 0.35 * rfmc["F점수"] + 0.62* rfmc["R점수"]) * 100

In [198]:
rfmc.head()

,주문자ID번호,M점수,F점수,R점수,RFM점수,W_RFM점수
0,rain3770010-3298-3375,1,2,2,5,197.0
1,sominnara010-6505-6625,1,1,1,3,100.0
2,33127561@n010-6857-8723,1,2,1,4,135.0
3,70359972@n010-3080-0528,1,3,2,6,232.0
4,silentna89010-6315-0720,1,3,5,9,418.0


#### float타입 int로 바꿔주고 object를 숫자형 타입으로 바꿔주기

In [199]:
rfmc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4774 entries, 0 to 4773
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   주문자ID번호  4774 non-null   object 
 1   M점수      4774 non-null   int64  
 2   F점수      4774 non-null   int64  
 3   R점수      4774 non-null   int64  
 4   RFM점수    4774 non-null   int64  
 5   W_RFM점수  4774 non-null   float64
dtypes: float64(1), int64(4), object(1)
memory usage: 261.1+ KB


In [200]:
rfmc.loc[:,'주문자ID번호'] = rfmc.loc[:,'주문자ID번호'].astype('category').cat.codes

In [201]:
rfmc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4774 entries, 0 to 4773
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   주문자ID번호  4774 non-null   int16  
 1   M점수      4774 non-null   int64  
 2   F점수      4774 non-null   int64  
 3   R점수      4774 non-null   int64  
 4   RFM점수    4774 non-null   int64  
 5   W_RFM점수  4774 non-null   float64
dtypes: float64(1), int16(1), int64(4)
memory usage: 233.1 KB


In [202]:
rfmc['W_RFM점수']=rfmc['W_RFM점수'].astype('int')
rfmc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4774 entries, 0 to 4773
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   주문자ID번호  4774 non-null   int16
 1   M점수      4774 non-null   int64
 2   F점수      4774 non-null   int64
 3   R점수      4774 non-null   int64
 4   RFM점수    4774 non-null   int64
 5   W_RFM점수  4774 non-null   int32
dtypes: int16(1), int32(1), int64(4)
memory usage: 214.5 KB


### 1. RandomForestRegression

In [203]:
# train_test_split으로 데이터 분리
x=rfmc.drop(columns=['W_RFM점수'])
y=rfmc['W_RFM점수']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [204]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((3819, 5), (955, 5), (3819,), (955,))

In [205]:
# RandomForest 회귀 사용해서 절대오차 구하기
rg=RandomForestRegressor()
rg.fit(x_train,y_train)
prediction=rg.predict(x_test)
rg_mae=mean_absolute_error(y_test,prediction)

In [206]:
rg_mae

0.023015706806282652

두번째 부터는 train_test_split부터 진행 (이미 되어있음)

### 2. Logistic Regression

In [207]:
rfmc.head()

,주문자ID번호,M점수,F점수,R점수,RFM점수,W_RFM점수
0,2838,1,2,2,5,197
1,3024,1,1,1,3,100
2,1358,1,2,1,4,135
3,1528,1,3,2,6,231
4,2966,1,3,5,9,418


In [208]:
# rfmc=RFM.copy()
# rfmc["W_RFM점수"]  = (0.5 * rfmc["M점수"] + 0.3 * rfmc["F점수"] + 0.2* rfmc["R점수"]) * 10
# rfmc.loc[:,'주문자ID번호'] = rfmc.loc[:,'주문자ID번호'].astype('category').cat.codes
# rfmc.info()


In [209]:
# rfmc['W_RFM점수']=rfmc['W_RFM점수'].astype('int')
# rfmc.info()

In [210]:
from sklearn.model_selection import train_test_split
x=rfmc.drop(columns=['W_RFM점수'])
y=rfmc['W_RFM점수']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [211]:
lr=LogisticRegression()
lr.fit(x_train,y_train)
prediction=lr.predict(x_test)
lr_mae=mean_absolute_error(y_test,prediction)
lr_mae

C:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


93.83979057591623

### 3. Linear Regression

In [212]:
lnr=LinearRegression()
lnr.fit(x_train,y_train)
prediction=lnr.predict(x_test)
lnr_mae=mean_absolute_error(y_test,prediction)
lnr_mae

0.3187314274082809

Logistic은 값이 너무 크게 나와서 제외하고 비교

In [213]:
rg_mae,lnr_mae

(0.023015706806282652, 0.3187314274082809)

1분기 분석은 완료했다, 절대오차로 비교한 결과 RandomForest가 가장 정확도가 높은 것으로 확인 되었다.

이와 같은 방법으로 2분기 데이터도 비교해보려고 한다

In [214]:
rfmc2=RFM2.copy()

In [215]:
rfmc2["W_RFM점수"]  = (0 * rfmc2["M점수"] + 0.39 * rfmc2["F점수"] + 0.61 * rfmc2["R점수"]) * 100

In [216]:
rfmc2.head()

,주문자ID번호,R점수,F점수,M점수,RFM점수,W_RFM점수
0,6773359@n010-9381-5574,5,3,4,12,422.0
1,49938077@n010-8288-1034,5,3,3,11,422.0
2,plom90010-4209-3577,5,4,4,13,461.0
3,010-2926-1299,5,3,1,9,422.0
4,dlagmltmd95010-3205-7859,5,2,2,9,383.0


In [217]:
rfmc2.loc[:,'주문자ID번호'] = rfmc2.loc[:,'주문자ID번호'].astype('category').cat.codes
rfmc2['W_RFM점수']=rfmc2['W_RFM점수'].astype('int')
rfmc2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4774 entries, 0 to 4773
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   주문자ID번호  4774 non-null   int16
 1   R점수      4774 non-null   int64
 2   F점수      4774 non-null   int64
 3   M점수      4774 non-null   int64
 4   RFM점수    4774 non-null   int64
 5   W_RFM점수  4774 non-null   int32
dtypes: int16(1), int32(1), int64(4)
memory usage: 214.5 KB


In [218]:
# train_test_split으로 데이터 분리
x=rfmc2.drop(columns=['W_RFM점수'])
y=rfmc2['W_RFM점수']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

### 1. RandomForestRegressor

In [219]:
rg=RandomForestRegressor()
rg.fit(x_train,y_train)
prediction=rg.predict(x_test)
rg_mae=mean_absolute_error(y_test,prediction)
rg_mae

0.10043979057591632

In [220]:
rfmc2.head()

,주문자ID번호,R점수,F점수,M점수,RFM점수,W_RFM점수
0,1519,5,3,4,12,422
1,1442,5,3,3,11,422
2,2766,5,4,4,13,460
3,145,5,3,1,9,422
4,1930,5,2,2,9,383


In [221]:
x=rfmc2.drop(columns=['W_RFM점수'])
y=rfmc2['W_RFM점수']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

### 2. LogisticRegression

In [222]:
lr=LogisticRegression()
lr.fit(x_train,y_train)
prediction=lr.predict(x_test)
lr_mae=mean_absolute_error(y_test,prediction)
lr_mae

C:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


108.45340314136125

### 3. LinearRegression

In [223]:
lnr=LinearRegression()
lnr.fit(x_train,y_train)
prediction=lnr.predict(x_test)
lnr_mae=mean_absolute_error(y_test,prediction)
lnr_mae

0.18829495783244457

In [224]:
rg_mae,lnr_mae

(0.10043979057591632, 0.18829495783244457)

# 결과

평균 절대 오차가 매우 낮은 것으로 확인했기 때문에, W_RFM점수를 원본 데이터에다가도 옮겨준다

In [225]:
RFM["W_RFM점수"]  = (0.03 * RFM["M점수"] + 0.35 * RFM["F점수"] + 0.62* RFM["R점수"]) * 100
RFM.head()

,주문자ID번호,M점수,F점수,R점수,RFM점수,W_RFM점수
0,rain3770010-3298-3375,1,2,2,5,197.0
1,sominnara010-6505-6625,1,1,1,3,100.0
2,33127561@n010-6857-8723,1,2,1,4,135.0
3,70359972@n010-3080-0528,1,3,2,6,232.0
4,silentna89010-6315-0720,1,3,5,9,418.0


In [226]:
RFM2["W_RFM점수"]  = (0.03 * RFM2["M점수"] + 0.35 * RFM2["F점수"] + 0.62* RFM2["R점수"]) * 100
RFM2.head()

,주문자ID번호,R점수,F점수,M점수,RFM점수,W_RFM점수
0,6773359@n010-9381-5574,5,3,4,12,427.0
1,49938077@n010-8288-1034,5,3,3,11,424.0
2,plom90010-4209-3577,5,4,4,13,462.0
3,010-2926-1299,5,3,1,9,418.0
4,dlagmltmd95010-3205-7859,5,2,2,9,386.0
